### Shared Object ###

This is the usal way to import the BondMachine toolkit into your python environment.

In [ ]:
import os
BONDMACHINE_DIR="/home/"+os.environ["USER"]+"/bin"
os.environ["BONDMACHINE_DIR"]=BONDMACHINE_DIR
os.environ["PATH"]=os.environ["PATH"]+":"+os.environ["BONDMACHINE_DIR"]

In [ ]:
os.environ['XILINX_HLS'] = '/tools/Xilinx/Vitis_HLS/2023.2'
os.environ['XILINX_VIVADO'] = '/tools/Xilinx/Vivado/2023.2'
os.environ['XILINX_VITIS'] = '/tools/Xilinx/Vitis/2023.2'
os.environ['PATH']=os.environ["PATH"]+":"+os.environ['XILINX_HLS']+"/bin:"+os.environ['XILINX_VIVADO']+"/bin:"+os.environ['XILINX_VITIS']+"/bin:"

Let's import also some other useful libraries as display.

In [ ]:
from IPython import display

In [ ]:
!bmhelper doctor

Now let's write down a small segment of assembly code. It is a simple program that gets the value from a shared object and prints it on the output.


In [ ]:
%%bash
cat <<EOF > program.asm
clr r0
lfsr82r r0 lfsr80
r2o r0 o0
j 1
EOF

Here's a table with the instructions used in the assembly code and their meanings:

| Instruction | Meaning |
|-------------|---------|
| `clr r0`    | Clear the value in register `r0` |
| `lfsr82r r0 lfsr80` | Load the value from the shared object `lfsr80` into register `r0` |
| `r2o r0 o0` | Move the value from register `r0` to output `o0` |
| `j 1` | Jump to the instruction at address `1` |

Like in the previous example, we can use the `procbuilder` to create a processor and load the assembly code into it. By specifying the `shared_constraints` parameter, we can also say to the processor that there will be a shared object called `lfsr80` that will be used by the program.

In [ ]:
%%bash
procbuilder -register-size 8 -registers 2 -inputs 0 -outputs 1 -opcodes clr,inc,lfsr82r,r2o,j  -save-machine processor.json -input-assembly program.asm -shared-constraints "lfsr8:34"

Let's see the processor we just created:

In [ ]:
!ls

Let's instantiate a new BondMachine.

In [ ]:
!bondmachine -bondmachine-file bondmachine.json

Add the processor to the BondMachine.

In [ ]:
!bondmachine -bondmachine-file bondmachine.json -add-domains processor.json
!bondmachine -bondmachine-file bondmachine.json -add-processor 0

See the BondMachine.

In [ ]:
!bondmachine -bondmachine-file bondmachine.json -emit-dot -dot-detail 5 | dot -Tpng > bondmachine.png
display.Image("bondmachine.png")

Let's add an output to the BondMachine.

In [ ]:
!bondmachine -bondmachine-file bondmachine.json -add-outputs 1

In [ ]:
!bondmachine -bondmachine-file bondmachine.json -emit-dot -dot-detail 5 | dot -Tpng > bondmachine.png
display.Image("bondmachine.png")

Let's connect the output to the processor.

In [ ]:

!bondmachine -bondmachine-file bondmachine.json -add-bond p0o0,o0

In [ ]:
!bondmachine -bondmachine-file bondmachine.json -emit-dot -dot-detail 5 | dot -Tpng > bondmachine.png
display.Image("bondmachine.png")

Let's add the shared object to the BondMachine. It is a linear feedback shift register (LFSR 8 bits) that will be used by the processor.

In [ ]:
!bondmachine -bondmachine-file bondmachine.json -add-shared-objects "lfsr8:34"

Let's see how the BondMachine looks like now.

In [ ]:
!bondmachine -bondmachine-file bondmachine.json -emit-dot -dot-detail 5 | dot -Tpng > bondmachine.png
display.Image("bondmachine.png")

Let's connect the shared object to the processor.

In [ ]:
!bondmachine -bondmachine-file bondmachine.json -connect-processor-shared-object 0,0

In [ ]:
!bondmachine -bondmachine-file bondmachine.json -emit-dot -dot-detail 5 | dot -Tpng > bondmachine.png
display.Image("bondmachine.png")

In the end we can see the BondMachine JSON description.

In [ ]:
!cat bondmachine.json

... and launch the synthesis ... implementation ... and programming.

In [ ]:
#make project, ...., program